In [3]:
# Required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-3-5d8b9e5642ff>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-5d8b9e5642ff>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
#spacex_plotly_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
#spacex_plotly_data.to_csv('spacex_plotly_data.csv',index=False)
#!curl "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [4]:
# read data
spacex_df = pd.read_csv('spacex_plotly_data.csv')
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
# Create dictionary for launch site values
sites = []
sites.append({'label': 'All Sites', 'value': 'All Sites'})

# fill values in dictionary
for location in uniquelaunchsites:
    sites.append({'label': location, 'value': location})

sites

[{'label': 'All Sites', 'value': 'All Sites'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [6]:
#!curl "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [7]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [ ]:
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#0e1012',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=sites,
                                            value='All Sites',
                                            placeholder="Select a Launch Site here", 
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                7500:'7500', 10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'All Sites':
        fig = px.pie(spacex_df, names='Launch Site', values='class' ,title='Launch Success for All Sites',template='plotly_dark')
        return fig
    else:
        #specific_df = spacex_df['Launch Site']
        site_df=spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(site_df, names='class',title='Launch Success for Site',template='plotly_dark')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])

def get_scatter_plot(entered_site, payload_slider):
    if entered_site == 'All Sites':
        payload_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])
                                 &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        
        scatterfig = px.scatter(payload_df, x="Payload Mass (kg)", y="class",
                                size = "Payload Mass (kg)",
                                marginal_x="histogram", marginal_y="rug",
                                color="Booster Version Category",
                                template='plotly_dark')
        return scatterfig
    else:
        site_df=spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        payload_df = site_df[(site_df['Payload Mass (kg)']>=payload_slider[0]) #  (minimum payload) gets all payload data points greater than or equal to selected range
                          &(spacex_df['Payload Mass (kg)']<=payload_slider[1])] # (maximum payload) less than or equal to
            
        scatterfig = px.scatter(payload_df, x="Payload Mass (kg)", y="class",
                                size = "Payload Mass (kg)", 
                                marginal_x="histogram", marginal_y="rug",
                                color="Booster Version Category",
                                template='plotly_dark')
        return scatterfig
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1633455707.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1633455707.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1633455707.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1633455707.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1633455707.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1633455707.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 18:26:17] "GET /_dash-component-suites/d